In [14]:
from sklearn import datasets

n = 5
x_train = [0]*n
x_test =[0]*n
y_train = [0]*n
y_test = [0]*n

for i in range(0,n):
    data = datasets.make_friedman1(n_samples = 100, n_features = 5, noise = 1)
    x_train[i] = data[0][0:50]
    x_test[i] = data[0][50:data[0].shape[0]]
    y_train[i] = data[1][0:50]
    y_test[i] = data[1][50:data[1].shape[0]]
    
for i in range(0,n):
    print(y_test[i])
    print(len(y_test[i]))
    print('------------------------')

[16.83674331  5.0055445  14.65720872 14.32226773 13.67501285  9.00347718
 11.9676993  16.41679307  9.09017411 11.35392473 11.32990743 11.37964533
 16.45789052 12.25102207  8.38601901  5.66467488 13.93644077  9.72746907
 23.20671258 15.15021111 11.47506065 12.00749935 22.23030118 17.94513394
 12.80498989 19.38816163 18.94058155 15.03462384 21.73845161 13.91300025
  9.47070597 14.44240114 17.32648687 10.33521344 17.90126728 17.91402086
 22.55214221  7.28722661 19.62909647  8.0961218  11.00308397 10.08794414
 21.62217803 16.06166059  8.35304334 19.24328934  9.18547369 13.52570868
 18.26947871 13.45236333]
50
------------------------
[21.39216847 11.23245745  8.23288969 16.36011289 18.06378956 18.81450585
 13.27238438 14.3461147   6.94931068  9.85851779 19.44288504 22.64137356
 15.35376564 11.47946004  9.99737608 20.04394441  9.63839456 19.27625458
 15.56288633 13.47441268 12.8178688  14.0778032  10.89732009 21.84944397
 14.09366538 20.98731739 17.44615546  8.63027394 18.54201049 20.949642

In [22]:
#compare grid and random search. search over all combinations. random forest

'''
note that, for random forest:
n_estimators = num.trees
min_samples_leaf = min.node.size
max_features = mtry
max_samples = sample.fraction
'''

import math
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

par_grid = {'n_estimators':[1,2,3,4,5], 
        'min_samples_leaf':np.linspace(0.1,0.5,5), 
        'max_features':np.linspace(0.1,0.5,5), 
        'max_samples':np.linspace(0.1,0.5,5)}

results_rs = pd.DataFrame(np.zeros((n,4)), columns = ['out-score', 'in-score', 'time', 'params'])
results_gs = pd.DataFrame(np.zeros((n,4)), columns = ['out-score', 'in-score', 'time', 'params'])

for i in range(0,n):
    gs = model_selection.GridSearchCV(estimator = RandomForestRegressor(), 
                                      param_grid = par_grid, 
                                      scoring = 'neg_root_mean_squared_error', 
                                      cv = 3)
    
    start = time.thread_time()
    gs.fit(x_train[i], y_train[i])
    stop = time.thread_time()
    
    results_gs.loc[i, 'params'] = str(gs.best_params_)
    results_gs.loc[i, 'in-score'] = - gs.best_score_
    gs_pred_i = gs.best_estimator_.predict(x_test[i])
    results_gs.loc[i, 'out-score'] = math.sqrt(mean_squared_error(y_true = np.array(y_test[i]), y_pred = gs_pred_i))
    results_gs.loc[i, 'time'] = stop - start

    rs = model_selection.RandomizedSearchCV(estimator = RandomForestRegressor(), 
                                        param_distributions = par_grid,
                                        n_iter = 5**4,
                                        scoring = 'neg_root_mean_squared_error',
                                        cv = 3)
    
    start = time.thread_time()
    rs.fit(x_train[i], y_train[i])
    stop = time.thread_time()
    
    results_rs.loc[i, 'params'] = str(rs.best_params_)
    results_rs.loc[i, 'in-score'] = - rs.best_score_
    rs_pred_i = rs.best_estimator_.predict(x_test[i])
    results_rs.loc[i, 'out-score'] = math.sqrt(mean_squared_error(y_true = np.array(y_test[i]), y_pred = rs_pred_i))
    results_rs.loc[i, 'time'] = stop - start
    
print(results_gs)
print(results_rs)


<class 'list'>
(50,)
<class 'list'>
(50,)
<class 'list'>
(50,)
<class 'list'>
(50,)
<class 'list'>
(50,)
   out-score  in-score       time  \
0   4.010657 -3.206412  13.171875   
1   3.789346 -3.579530  12.546875   
2   3.734706 -3.891487  12.921875   
3   4.273810 -3.916630  12.703125   
4   4.850144 -3.795281  12.609375   

                                              params  
0  {'max_features': 0.4, 'max_samples': 0.5, 'min...  
1  {'max_features': 0.5, 'max_samples': 0.5, 'min...  
2  {'max_features': 0.4, 'max_samples': 0.5, 'min...  
3  {'max_features': 0.1, 'max_samples': 0.5, 'min...  
4  {'max_features': 0.4, 'max_samples': 0.4, 'min...  
   out-score  in-score       time  \
0   3.986194 -3.357438  12.656250   
1   4.523908 -3.794931  13.171875   
2   3.293583 -4.250355  12.875000   
3   3.635983 -3.948956  13.062500   
4   5.173235 -3.929870  12.875000   

                                              params  
0  {'n_estimators': 5, 'min_samples_leaf': 0.1, '...  
1  {'n_es

In [18]:
#compare grid and random search. search along only 1 parameter at a time. random forest.
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
import time

x = datasets.make_friedman1(n_samples = 50, n_features = 5, noise = 0)

#I decided to define the default parameters as somewhere in the middle of the entire space
def_pars = {'n_estimators':3, 'min_samples_leaf':0.3, 'max_samples':0.3, 'max_features':0.3}
best_pars_gs = def_pars
best_pars_rs = def_pars

param_grid = {'n_estimators':[1,2,3,4,5], 
        'min_samples_leaf':np.linspace(0.1,0.5,5), 
        'max_features':np.linspace(0.1,0.5,5), 
        'max_samples':np.linspace(0.1,0.5,5)}
space_sizes = {'n_estimators':5, 'min_samples_leaf':5, 'max_features':5, 'max_samples':5}

for par in param_grid:
    rf = RandomForestRegressor(n_estimators = def_pars['n_estimators'],
                                min_samples_leaf = def_pars['min_samples_leaf'],
                                max_samples = def_pars['max_samples'],
                                max_features = def_pars['max_features']
                               )
    one_par = {par:param_grid[par]}
    gs = model_selection.GridSearchCV(estimator = rf,
                                      param_grid = one_par,
                                      cv = 3,
                                      scoring = 'neg_mean_squared_error'
                                     )
    gs.fit(x[0], x[1])
    best_pars_gs[par] = gs.best_params_[par]
    rs = model_selection.RandomizedSearchCV(estimator = rf,
                                           param_distributions = one_par,
                                           cv = 3,
                                            n_iter = space_sizes[par],
                                           scoring = 'neg_mean_squared_error')
    rs.fit(x[0], x[1])
    best_pars_rs[par] = rs.best_params_[par]

print(best_pars_gs)
print(best_pars_rs)
    

{'n_estimators': 3, 'min_samples_leaf': 0.4, 'max_samples': 0.5, 'max_features': 0.30000000000000004}
{'n_estimators': 3, 'min_samples_leaf': 0.4, 'max_samples': 0.5, 'max_features': 0.30000000000000004}
